In [2]:
import sys
sys.path.insert(1, '../../scripts/')
from s3_support import *
%matplotlib inline

# load data

In [3]:
q = '''select * from form where type=3'''
forms_table = redshift_query_read(q, schema='production')

In [4]:
forms_table.head(2)

,id,org,datecreated,datelive,path,status,type,template,enableenddate,enablestartdate,recurringplanenddaterequired,categorization,isvirtual
0,1637,1648,NaT,2011-11-10 11:27:42,andhob,11,3,9,True,False,False,0,0
1,1782,53,2012-01-13 09:44:43,2007-09-10 00:00:00,nubawh,11,3,9,True,False,False,0,0


In [5]:
forms_table['isvirtual'].value_counts()

0    13620
1      535
Name: isvirtual, dtype: int64

In [6]:
forms_table['categorization'].value_counts()

0     10370
8       843
6       639
1       624
5       387
9       329
3       188
4       147
11      144
7       143
15      101
2        97
12       48
14       46
13       26
10       23
Name: categorization, dtype: int64

## processing

In [7]:
q = '''select
            t.org,
            t.form,
            date_trunc('week', t.date) as week,
            count(t.id) as trans_count,
            sum(t.amount) as trans_vol,
            sum(t.donations_count) as donations_count,
            sum(t.donations_amt) as donations_vol,
            sum(t.purchases_count) as purchases_count,
            sum(t.purchases_amt) as purchases_vol,
            sum(t.registrations_count) as registrations_count,
            sum(t.registrations_amt) as registrations_vol,
            sum(t.events_amt) as events_vol,
            sum(t.events_count) as events_count
        from transactions as t
            left join form as f on f.id=t.form
        where
            t.status='A' and
            f.type=3
        group by date_trunc('week', t.date), t.form, t.org'''
trans = redshift_query_read(q, schema='production')

## p2p 

In [8]:
q = "select * from analyticsp2p_weekly"
df = redshift_query_read(q, schema='public')

## merging

In [9]:
# $41k floor for top 10 volume
top_10_vol_floor = 41000

form_totals = trans.groupby(['form', 'org'])[['trans_vol', 'donations_vol', 'purchases_count', 'purchases_vol', 'registrations_count', 'registrations_vol']].sum().reset_index()
form_totals['is_top_performer'] = form_totals['trans_vol']>top_10_vol_floor
top_performers = form_totals[form_totals['is_top_performer']]['form'].tolist()

In [10]:
trans['is_top_performer'] = trans['form'].isin(top_performers)
df['is_top_performer'] = df['form'].isin(top_performers)

## top performers

top 10% processing

In [11]:
buckets = [(0, 250), (250, 500), (500, 1000), (1000, 2500), 
           (2500, 5000), (5000, 10000), (10000, 20000), 
           (20000, 50000), (50000, 100000), (100000)]
for b in buckets:
    try:
        len_forms = len(form_totals[(form_totals['trans_vol']>=b[0])&(form_totals['trans_vol']<b[1])])
    except:
        len_forms = len(form_totals[form_totals['trans_vol']>=b])
        
    perc_forms = (len_forms / len(form_totals)) * 100.
    print("{}: {:,} forms ({:.2f}%)".format(b, len_forms, perc_forms))

(0, 250): 1,182 forms (13.15%)
(250, 500): 401 forms (4.46%)
(500, 1000): 608 forms (6.77%)
(1000, 2500): 1,170 forms (13.02%)
(2500, 5000): 1,159 forms (12.90%)
(5000, 10000): 1,260 forms (14.02%)
(10000, 20000): 1,284 forms (14.29%)
(20000, 50000): 1,188 forms (13.22%)
(50000, 100000): 472 forms (5.25%)
100000: 262 forms (2.92%)


In [12]:
# top 10%
len_all = len(form_totals)
top_10perc = int(len_all/10)

top_10_total_vol = form_totals.sort_values('trans_vol', ascending=False).head(top_10perc)

print("Top 10% total volume:")
print("{:,} forms".format(len(top_10_total_vol)))
print("${:,.2f} median volume".format(top_10_total_vol['trans_vol'].median()))
print("${:,.2f} mean volume".format(top_10_total_vol['trans_vol'].mean()))
print("Volume range: ${:,.2f} to ${:,.2f}".format(top_10_total_vol['trans_vol'].min(), top_10_total_vol['trans_vol'].max()))
print()
print("{:,.2f} mean registrants".format(top_10_total_vol['registrations_count'].mean()))
print("{:,.2f} median registrants".format(top_10_total_vol['registrations_count'].median()))
print("registrants range: {:,} to {:,}".format(top_10_total_vol['registrations_count'].min(), top_10_total_vol['registrations_count'].max()))

Top 10% total volume:
898 forms
$69,161.88 median volume
$113,641.10 mean volume
Volume range: $42,785.78 to $1,997,325.60

449.89 mean registrants
191.00 median registrants
registrants range: 0 to 10,032


### not top performers

In [13]:
# not top 10%
len_all = len(form_totals)
bottom_90perc = len_all - top_10perc

not_top_10_total_vol = form_totals.sort_values('trans_vol', ascending=False).tail(bottom_90perc)

print("Not top 10% total volume:")
print("{:,} forms".format(len(not_top_10_total_vol)))
print("${:,.2f} median volume".format(not_top_10_total_vol['trans_vol'].median()))
print("${:,.2f} mean volume".format(not_top_10_total_vol['trans_vol'].mean()))
print("Volume range: ${:,.2f} to ${:,.2f}".format(not_top_10_total_vol['trans_vol'].min(), not_top_10_total_vol['trans_vol'].max()))
print()
print("{:,.2f} mean registrants".format(not_top_10_total_vol['registrations_count'].mean()))
print("{:,.2f} median registrants".format(not_top_10_total_vol['registrations_count'].median()))
print("registrants range: {:,} to {:,}".format(not_top_10_total_vol['registrations_count'].min(), not_top_10_total_vol['registrations_count'].max()))

Not top 10% total volume:
8,088 forms
$3,821.56 median volume
$7,942.56 mean volume
Volume range: $0.00 to $42,783.67

51.78 mean registrants
11.00 median registrants
registrants range: 0 to 2,643


## underperformers

bottom 10%

In [14]:
# top 10%
len_all = len(form_totals)
bottom_10perc = int(len_all/10)

bottom_10_total_vol = form_totals.sort_values('trans_vol', ascending=True).head(bottom_10perc)

print("Bottom 10% total volume:")
print("{:,} forms".format(len(bottom_10_total_vol)))
print("${:,.2f} median volume".format(bottom_10_total_vol['trans_vol'].median()))
print("${:,.2f} mean volume".format(bottom_10_total_vol['trans_vol'].mean()))
print("Volume range: ${:,.2f} to ${:,.2f}".format(bottom_10_total_vol['trans_vol'].min(), bottom_10_total_vol['trans_vol'].max()))
print()
print("{:,.2f} mean registrants".format(bottom_10_total_vol['registrations_count'].mean()))
print("{:,.2f} median registrants".format(bottom_10_total_vol['registrations_count'].median()))
print("registrants range: {:,} to {:,}".format(bottom_10_total_vol['registrations_count'].min(), bottom_10_total_vol['registrations_count'].max()))

Bottom 10% total volume:
898 forms
$4.50 median volume
$27.31 mean volume
Volume range: $0.00 to $110.00

5.46 mean registrants
1.00 median registrants
registrants range: 0 to 385


In [15]:
form_totals['is_bottom_performer'] = form_totals['form'].isin(bottom_10_total_vol['form'].tolist())

# org conistency

are over performers and under performers consistent within a given org? can we assume that if an org has performed within a certain category in the past that their future events will perform within the same category?

In [16]:
org_consistency = form_totals.groupby('org')[['is_top_performer', 'is_bottom_performer']].median().reset_index()
org_consistency[['is_top_performer', 'is_bottom_performer']].value_counts()

is_top_performer  is_bottom_performer
0.0               0.0                    1044
1.0               0.0                     151
0.0               1.0                      91
0.5               0.0                      35
0.0               0.5                      25
0.5               0.5                       3
dtype: int64

the values appear to be very consistent with only 61 org's having a value that's not 1.0 or 0.0

In [17]:
org_counts = form_totals.groupby('org')['is_top_performer'].count().reset_index()
org_counts.columns = ['org', 'event_count']

e_cs = [(1), (2), (3, 5), (5)]
print("Events per org")
print("-"*40)
print("Orgs with 1 event: {:,}".format(len(org_counts[org_counts['event_count']==1])))
print("Orgs with 2 events: {:,}".format(len(org_counts[org_counts['event_count']==2])))
print("Orgs with 3 to 5 events: {:,}".format(len(org_counts[(org_counts['event_count']>=3)&(org_counts['event_count']<=5)])))
print("Orgs with 6+ events: {:,}".format(len(org_counts[org_counts['event_count']>=6])))

print()
print("Orgs with more than 1 event: {:,}".format(len(org_counts[org_counts['event_count']>=2])))

Events per org
----------------------------------------
Orgs with 1 event: 480
Orgs with 2 events: 241
Orgs with 3 to 5 events: 304
Orgs with 6+ events: 324

Orgs with more than 1 event: 869


There __are only 61 orgs with variable performance categories across events__ and __there are 835 orgs with more than 1 event__. This is solid evidence of consistency of performance category across multiple events as only 7% have changed categories between events.

# settings

In [18]:
aggs = {
    'reg_count': 'sum',
    'sub_reg_count': 'sum',
    'teams_count': 'sum',
    'reg_volume': 'sum',
    'don_volume': 'sum',
    'don_count': 'sum',
    'class_count': 'mean',
    'cat_count': 'mean',
    'promo_count': 'mean',
    'rest_count': 'mean',
    'amt_count': 'mean',
    'ded_count': 'mean',
    'fields': 'mean',
    'opt_fields': 'mean',
    'req_fields': 'mean',
    'allows_reg_ind': 'mean',
    'allows_teams': 'mean',
    'allows_reg_team_create': 'mean',
    'allows_sub_reg': 'mean',
    'allows_sub_reg_pfp': 'mean',
    'allows_other_don_amt': 'mean',
    'allows_pfp_off_don': 'mean',
    'allows_tfp_off_don': 'mean',
    'share_tfp': 'mean',
    'share_therm': 'mean',
    'share_donation': 'mean',
    'allows_social': 'mean',
    'social_templt_count': 'mean',
    'social_auto': 'mean',
    'pcnt_posts': 'mean',
    'mon_posts': 'mean',
    'count_posts': 'sum',
    'date_posts': 'mean',
    'email_templt_count': 'mean',
    'sponsors_count': 'mean'
}
event_stats = df.groupby('form').agg(aggs).reset_index()

In [19]:
event_stats['is_top_performer'] = event_stats['form'].isin(top_10_total_vol['form'].tolist())
event_stats['is_bottom_performer'] = event_stats['form'].isin(bottom_10_total_vol['form'].tolist())

In [21]:
pd.set_option('display.max_rows', 100)

tbl = event_stats.groupby(['is_top_performer', 'is_bottom_performer']).agg(['mean', 'median']).transpose()
tbl.columns = ['middle 80%', 'bottom 10%', 'top 10%']
tbl

middle 80%     bottom 10%        top 10%
form                   mean    904990.489101  903457.090305  920687.985092
                       median  954434.500000  953918.000000  958414.500000
reg_count              mean         0.000000       0.000000       0.000000
                       median       0.000000       0.000000       0.000000
sub_reg_count          mean         2.405189       0.585657      23.497706
                       median       0.000000       0.000000       0.000000
teams_count            mean         3.459105       0.633466      31.036697
                       median       0.000000       0.000000       5.000000
reg_volume             mean         0.000000       0.000000       0.000000
                       median       0.000000       0.000000       0.000000
don_volume             mean      3602.383587      18.082151   76079.895161
                       median       0.000000       0.000000   49028.845000
don_count              mean        38.945120       0.602922     682.220183
                       median       0.000000       0.000000     360.000000
class_count            mean         0.994210       0.770868       4.335446
                       median       0.000000       0.000000       0.000000
cat_count              mean         1.635547       1.532219       2.846054
                       median       1.000000       1.000000       2.000000
promo_count            mean         0.681739       0.244011       4.370436
                       median       0.000000       0.000000       0.000000
rest_count             mean         0.110493       0.091793       0.167015
                       median       0.000000       0.000000       0.000000
amt_count              mean         4.355738       4.233806       5.027561
                       median       4.000000       4.000000       4.980879
ded_count              mean         0.479170       0.409600       0.751543
                       median       0.000000       0.000000       0.000000
fields                 mean         0.872400       0.832031       2.601329
                       median       0.000000       0.000000       1.168990
opt_fields             mean         0.332115       0.282835       1.003131
                       median       0.000000       0.000000       0.000000
req_fields             mean         0.539714       0.548941       1.596318
                       median       0.000000       0.000000       0.975917
allows_reg_ind         mean         0.973920       0.742426       0.838450
                       median       1.000000       1.000000       1.000000
allows_teams           mean         0.934456       0.627426       0.846231
                       median       1.000000       1.000000       1.000000
allows_reg_team_create mean         0.660806       0.668527       0.783595
                       median       1.000000       1.000000       1.000000
allows_sub_reg         mean         0.995756       0.996016       0.996560
                       median       1.000000       1.000000       1.000000
allows_sub_reg_pfp     mean         0.106066       0.055864       0.026554
                       median       0.000000       0.000000       0.000000
allows_other_don_amt   mean         0.999325       1.000000       1.000000
                       median       1.000000       1.000000       1.000000
allows_pfp_off_don     mean         0.780566       0.741512       0.722876
                       median       1.000000       1.000000       1.000000
allows_tfp_off_don     mean         0.780566       0.741512       0.722876
                       median       1.000000       1.000000       1.000000
share_tfp              mean         0.000000       0.000000       0.000000
                       median       0.000000       0.000000       0.000000
share_therm            mean         0.000000       0.000000       0.000000
                       median       0.000000       0.000000       0.000000
share_donation         mean         0.000000       0.000000

### settings by event category

In [22]:
def form_cat(form):
    if form in forms_table['id'].tolist():
        return forms_table[forms_table['id']==form]['categorization'].iloc[0]
    return 0

event_stats['category'] = event_stats['form'].apply(form_cat)

In [23]:
event_stats.groupby(['category', 'is_top_performer', 'is_bottom_performer']).median().transpose()

category                      0                                       1   \
is_top_performer           False                     True          False   
is_bottom_performer        False      True          False          False   
form                    947271.0  946487.0  943400.500000  984672.500000   
reg_count                    0.0       0.0       0.000000       0.000000   
sub_reg_count                0.0       0.0       0.000000       0.000000   
teams_count                  0.0       0.0       2.000000       1.000000   
reg_volume                   0.0       0.0       0.000000       0.000000   
don_volume                   0.0       0.0   44006.465000    1408.000000   
don_count                    0.0       0.0     314.500000      21.000000   
class_count                  0.0       0.0       0.000000       0.000000   
cat_count                    1.0       1.0       2.000000       2.000000   
promo_count                  0.0       0.0       0.000000       0.000000   
rest_count                   0.0       0.0       0.000000       0.000000   
amt_count                    4.0       4.0       4.950829       4.996269   
ded_count                    0.0       0.0       0.000000       0.000000   
fields                       0.0       0.0       1.112143       0.000000   
opt_fields                   0.0       0.0       0.000000       0.000000   
req_fields                   0.0       0.0       1.000000       0.000000   
allows_reg_ind               1.0       1.0       1.000000       1.000000   
allows_teams                 1.0       1.0       1.000000       1.000000   
allows_reg_team_create       1.0       1.0       1.000000       1.000000   
allows_sub_reg               1.0       1.0       1.000000       1.000000   
allows_sub_reg_pfp           0.0       0.0       0.000000       0.000000   
allows_other_don_amt         1.0       1.0       1.000000       1.000000   
allows_pfp_off_don           1.0       1.0       1.000000       1.000000   
allows_tfp_off_don           1.0       1.0       1.000000       1.000000   
share_tfp                    0.0       0.0       0.000000       0.000000   
share_therm                  0.0       0.0       0.000000       0.000000   
share_donation               0.0       0.0       0.000000       0.000000   
allows_social                0.0       0.0       0.000000       0.000000   
social_templt_count          0.0       0.0       0.000000       0.000000   
social_auto                  0.0       0.0       0.000000       0.000000   
pcnt_posts                   0.0       0.0       0.000000       0.000000   
mon_posts                    0.0       0.0       0.000000       0.000000   
count_posts                  0.0       0.0       0.000000       0.000000   
date_posts                   0.0       0.0       0.000000       0.000000   
email_templt_count           0.0       0.0       0.000000       0.000000   
sponsors_count               0.0       0.0       4.767314       4.000000   

category                                                    2                  \
is_top_performer                           True          False                  
is_bottom_performer         True          False          False           True   
form                    980278.0  981321.000000  984220.000000  973873.000000   
reg_count                    0.0       0.000000       0.000000       0.000000   
sub_reg_count                0.0      21.000000       0.000000       0.000000   
teams_count                  1.0      34.000000       0.000000       0.000000   
reg_volume                   0.0       0.000000       0.000000       0.000000   
don_volume                   0.0   58446.750000    1150.000000      65.000000   
don_count                    0.0     623.000000      11.000000       2.000000   
class_count                  0.0       0.000000       0.000000       0.500000   
cat_count                    2.0       2.500000       2.000000       1.000000   
promo_count                  0.0       0.000000       0.000000 

### form settings by isvirtual

In [24]:
def form_isvirtual(form):
    if form in forms_table['id'].tolist():
        return forms_table[forms_table['id']==form]['isvirtual'].iloc[0]
    return 0

event_stats['isvirtual'] = event_stats['form'].apply(form_isvirtual)

In [25]:
event_stats.groupby(['isvirtual', 'is_top_performer', 'is_bottom_performer']).median().transpose()

isvirtual                      0                                       1  \
is_top_performer           False                     True          False   
is_bottom_performer        False      True          False          False   
form                    953684.0  953606.0  956204.000000  982488.000000   
reg_count                    0.0       0.0       0.000000       0.000000   
sub_reg_count                0.0       0.0       0.000000       0.000000   
teams_count                  0.0       0.0       6.000000       0.000000   
reg_volume                   0.0       0.0       0.000000       0.000000   
don_volume                   0.0       0.0   48212.840000    2767.000000   
don_count                    0.0       0.0     359.000000      23.000000   
class_count                  0.0       0.0       0.000000       0.000000   
cat_count                    1.0       1.0       2.000000       1.852941   
promo_count                  0.0       0.0       0.000000       0.000000   
rest_count                   0.0       0.0       0.000000       0.000000   
amt_count                    4.0       4.0       4.981366       4.971429   
ded_count                    0.0       0.0       0.000000       0.000000   
fields                       0.0       0.0       1.322581       0.000000   
opt_fields                   0.0       0.0       0.000000       0.000000   
req_fields                   0.0       0.0       1.000000       0.000000   
allows_reg_ind               1.0       1.0       1.000000       1.000000   
allows_teams                 1.0       1.0       1.000000       1.000000   
allows_reg_team_create       1.0       1.0       1.000000       1.000000   
allows_sub_reg               1.0       1.0       1.000000       1.000000   
allows_sub_reg_pfp           0.0       0.0       0.000000       0.000000   
allows_other_don_amt         1.0       1.0       1.000000       1.000000   
allows_pfp_off_don           1.0       1.0       1.000000       1.000000   
allows_tfp_off_don           1.0       1.0       1.000000       1.000000   
share_tfp                    0.0       0.0       0.000000       0.000000   
share_therm                  0.0       0.0       0.000000       0.000000   
share_donation               0.0       0.0       0.000000       0.000000   
allows_social                0.0       0.0       0.000000       0.000000   
social_templt_count          0.0       0.0       0.000000       0.000000   
social_auto                  0.0       0.0       0.000000       0.000000   
pcnt_posts                   0.0       0.0       0.000000       0.000000   
mon_posts                    0.0       0.0       0.000000       0.000000   
count_posts                  0.0       0.0       0.000000       0.000000   
date_posts                   0.0       0.0       0.000000       0.000000   
email_templt_count           0.0       0.0       0.000000       0.000000   
sponsors_count               0.0       0.0       5.000000       0.000000   
category                     0.0       0.0       0.000000       7.000000   

isvirtual                                             
is_top_performer                                True  
is_bottom_performer              True          False  
form                    982957.000000  979981.000000  
reg_count                    0.000000       0.000000  
sub_reg_count                0.000000       0.000000  
teams_count                  0.000000       0.000000  
reg_volume                   0.000000       0.000000  
don_volume                   0.000000   59718.300000  
don_count                    0.000000     376.000000  
class_count                  0.000000       0.000000  
cat_count                    2.000000       1.933333  
promo_count                  0.000000       0.000000  
rest_count                   0.000000       0.000000  
amt_count                    5.000000       4.960000  
ded_count                    0.000000       0.000000  
fields                       0.000000       0.439024  
opt_fields              

# category representation by performance group

In [34]:
cat_counts = event_stats.groupby('category')['form'].nunique()

In [32]:
event_stats.groupby('category')['isvirtual', 'is_top_performer', 'is_bottom_performer'].mean().reset_index().merge(cat_counts, on='category')

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,category,isvirtual,is_top_performer,is_bottom_performer,form
0,0,0.000000,0.058791,0.062975,9083
1,1,0.078723,0.095745,0.036170,470
2,2,0.187500,0.087500,0.025000,80
3,3,0.197080,0.160584,0.036496,137
4,4,0.168317,0.198020,0.069307,101
5,5,0.064748,0.262590,0.028777,278
6,6,0.137868,0.139706,0.060662,544
7,7,0.166667,0.055556,0.101852,108
8,8,0.097633,0.048817,0.081361,676
9,9,0.316017,0.134199,0.077922,231
